<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-summary" data-toc-modified-id="Data-summary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="#section 1">Data summary</a></a></span><ul class="toc-item"><li><span><a href="#Group-the-behavioral-data-by-target-load-and-subjects" data-toc-modified-id="Group-the-behavioral-data-by-target-load-and-subjects-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Group the behavioral data by target load and subjects</a></span></li></ul></li><li><span><a href="#Consistencies-in-individual-differences-in-the-behavioral-data" data-toc-modified-id="Consistencies-in-individual-differences-in-the-behavioral-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="#section 2">Consistencies in individual differences in the behavioral data</a></a></span><ul class="toc-item"><li><span><a href="#Obtain-rankings-of-participants-across-the-120-trials" data-toc-modified-id="Obtain-rankings-of-participants-across-the-120-trials-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Obtain rankings of participants across the 120 trials</a></span></li><li><span><a href="#Check-for-consistencies-when-compared-across-target-load-sets-for-these-ranked-individuals" data-toc-modified-id="Check-for-consistencies-when-compared-across-target-load-sets-for-these-ranked-individuals-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Check for consistencies when compared across target load sets for these ranked individuals</a></span></li><li><span><a href="#Check-for-consistencies-when-compared-across-speeds-for-these-ranked-individuals" data-toc-modified-id="Check-for-consistencies-when-compared-across-speeds-for-these-ranked-individuals-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Check for consistencies when compared across speeds for these ranked individuals</a></span></li></ul></li><li><span><a href="#Analysis-using-number-of-targets-chosen" data-toc-modified-id="Analysis-using-number-of-targets-chosen-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="#section 3">Analysis using number of targets chosen</a></a></span><ul class="toc-item"><li><span><a href="#Group-the-behavioral-data-by-target-load" data-toc-modified-id="Group-the-behavioral-data-by-target-load-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Group the behavioral data by target load</a></span></li><li><span><a href="#Plot-the-grouped-data" data-toc-modified-id="Plot-the-grouped-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Plot the grouped data</a></span></li><li><span><a href="#Subject-wise-analysis-of-the-number-of-targets-chosen:--Appendix-to-Section-3" data-toc-modified-id="Subject-wise-analysis-of-the-number-of-targets-chosen:--Appendix-to-Section-3-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span><a href="#section 3.1">Subject wise analysis of the number of targets chosen:</a>  Appendix to <a href="#section 3">Section 3</a></a></span></li><li><span><a href="#Plot-the-subject-ranks-vs-avg.-number-of-targets-chosen-for-each-target-load" data-toc-modified-id="Plot-the-subject-ranks-vs-avg.-number-of-targets-chosen-for-each-target-load-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Plot the subject ranks vs avg. number of targets chosen for each target load</a></span></li></ul></li></ul></div>

In [1]:
# clear all the variables & load the necessary modules
%reset -f
import numpy as np
import sys
import turicreate as tc
import turicreate.aggregate as agg
import os
from os import walk
import fnmatch
import scipy.io
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from scipy.stats import norm
from statsmodels.formula.api import ols
from scipy.stats.mstats import zscore
import scipy.stats as st
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import random
from plotly import tools
# Path variable (where the data is located)
myPath = '../data/processed_files/MOT_csv_files/behavior_data/'#'./data/MOT_csv_files/behavior_data/'

In [2]:
behavior_data = tc.SFrame.read_csv(myPath + 'behavior_data.csv')
behavior_data.explore()

Finished parsing file /Volumes/My Passport/Github/MOT_bayesian_model/data/MOT_csv_files/behavior_data/behavior_data.csv

Parsing completed. Parsed 100 lines in 0.044287 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,int,int,int,int,float,int,int,int,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Volumes/My Passport/Github/MOT_bayesian_model/data/MOT_csv_files/behavior_data/behavior_data.csv

Parsing completed. Parsed 6000 lines in 0.034725 secs.

Materializing SFrame

In [3]:
len(behavior_data[behavior_data['subject_Num']==102]['trial_Num'].unique())

30

<a> id = section 1></a>
# [Data summary](#section 1) 

## Group the behavioral data by target load and subjects

In [4]:
subject_data = behavior_data.groupby(['num_targets','subject_Num'],operations = {'avg_num_targets_chosen':agg.MEAN('num_targets_chosen'),
                                                               'std_num_targets_chosen':agg.STD('num_targets_chosen'),
                                                                                'avg_accuracy':agg.MEAN('accuracy')})
subject_data = subject_data.sort(['subject_Num','num_targets'])

In [5]:
trial_data = behavior_data.groupby(['trial_Num_Original'],operations = {'avg_num_targets_chosen':agg.MEAN('num_targets_chosen'),
                                                               'std_num_targets_chosen':agg.STD('num_targets_chosen'),
                                                                                'avg_accuracy':agg.MEAN('accuracy'),
                                                                       'num_targets':agg.MEAN('num_targets')})
trial_data = trial_data.sort(['trial_Num_Original'])
trial_data.explore()

Materializing SFrame

In [6]:
accuracy_data = subject_data.groupby('num_targets',operations = {'avg_acc':agg.MEAN('avg_accuracy'),
                                                               'std_acc':agg.STD('avg_accuracy'),
                                                                 'Min_acc':agg.MIN('avg_accuracy'),
                                                                 'Max_acc':agg.MAX('avg_accuracy')})
median = []

for target in behavior_data['num_targets'].unique().sort():
    target_data = behavior_data[behavior_data['num_targets']==target]
    median.append(np.median(target_data['accuracy'].to_numpy()))

accuracy_data = accuracy_data.sort('num_targets')
accuracy_data['Median'] = median
accuracy_data['num_targets','avg_acc','Median','std_acc','Max_acc','Min_acc']

num_targets,avg_acc,Median,std_acc,Max_acc,Min_acc
3,0.9283407069999997,1.0,0.06202431569839687,1.0,0.71666665
4,0.8250000000000001,0.75,0.07830229881682914,0.975,0.6375
5,0.7207999999999999,0.8,0.06921965038917778,0.8199999999999998,0.59
6,0.6438366770000001,0.666667,0.06148760389650442,0.7834500000000002,0.47499994999999995
7,0.6215384129999999,0.571429,0.04835293750605936,0.7285716,0.5356499999999998
8,0.5715,0.625,0.04066171417931125,0.6624999999999999,0.46249999999999997


In [7]:
sec1_trace = []
for target in subject_data['num_targets'].unique().sort():
    target_data = subject_data[subject_data['num_targets']==target]
    #sec3_trace.append(go.Histogram(x = target_data['avg_accuracy'].to_numpy(),
    #                             histnorm = 'percent', name='Target load: ' + str(target)))
    sec1_trace.append(target_data['avg_accuracy'].to_numpy())

group_labels = ['3','4','5','6','7','8']
        
fig = ff.create_distplot(sec1_trace,group_labels, curve_type='kde', bin_size = accuracy_data['std_acc'].to_numpy())


fig['layout'].update(title='Hist and Rug plots of accuracy distribution for various target loads')
fig['layout']['xaxis'].update(title='Accuracy distribution for target loads')
fig['layout']['yaxis'].update(title='Frequency')

iplot(fig, filename='histograms of accuracies for various target loads')  

In [8]:
len(subject_data)/6

50.0

<a id = section 2></a>
# [Consistencies in individual differences in the behavioral data](#section 2) 

In this section, we check whether the individual differences we observe are consistent across the individuals across conditions. We check for the consistencies in the following ways:

* First, obtain the individual rankings of all the 50 participants by collapsing across 120 trials
* Second, check if these rankings are consistent when the compared across target load sets [3,5,7] vs. [4,6,8]
* Third, check if these rankings are consistent across speed conditions [2.8 8.4] vs. [5.6 11.2]


## Obtain rankings of participants across the 120 trials

In [9]:
individual_ranking = behavior_data.groupby('subject_Num',operations = {'avg_accuracy':agg.MEAN('accuracy')})
individual_ranking = individual_ranking.sort('avg_accuracy',ascending=False)
individual_ranking['rank'] = np.arange(1,len(individual_ranking)+1)
individual_ranking = individual_ranking.sort('subject_Num')
individual_ranking

subject_Num,avg_accuracy,rank
101,0.7477083333333335,16
102,0.8009250000000003,2
103,0.7633583333333335,10
104,0.7680833333333337,8
105,0.6633083333333334,43
106,0.7353749999999998,23
107,0.7586166666666667,12
108,0.7548249999999999,14
109,0.7060833333333331,30
110,0.6245749999999994,48


## Check for consistencies when compared across target load sets for these ranked individuals

In [10]:
odd_set = [6,7,8]
even_set = [3,4,5]
sub_id = individual_ranking['subject_Num'].to_numpy()

even_load_data = behavior_data.filter_by(even_set,'num_targets')
individual_even_load_data = even_load_data.groupby('subject_Num',operations = {'avg_accuracy':agg.MEAN('accuracy')})
individual_even_load_data = individual_even_load_data.sort('subject_Num')
individual_even_load_data['rank'] = individual_ranking['rank']
individual_even_load_data = individual_even_load_data.sort('rank')

odd_load_data = behavior_data.filter_by(odd_set,'num_targets')
individual_odd_load_data = odd_load_data.groupby('subject_Num',operations = {'avg_accuracy':agg.MEAN('accuracy')})
individual_odd_load_data = individual_odd_load_data.sort('subject_Num')
individual_odd_load_data['rank'] = individual_ranking['rank']
individual_odd_load_data = individual_odd_load_data.sort('rank')

hsb = individual_odd_load_data
hsb['avg_accuracy_even'] = individual_even_load_data['avg_accuracy']
hsb2 = hsb.to_dataframe()

mod = ols("avg_accuracy ~ avg_accuracy_even", data=hsb2)
res = mod.fit()
print(res.rsquared)

sec2_trace = go.Scatter(x = individual_even_load_data['avg_accuracy'].to_numpy(),
                       y = individual_odd_load_data['avg_accuracy'].to_numpy(),
                       mode='markers+text', marker = dict(size = 12, opacity = 0.65, color = 'gray',line = dict(width = 1.5, color='black')),
                       text = individual_even_load_data['rank'].to_numpy(),
                        textfont = dict(size = 7),
                       showlegend=False)

sec2_trace_fit = go.Scatter(x = np.linspace(individual_even_load_data['avg_accuracy'].min(),individual_even_load_data['avg_accuracy'].max(),100),
                   y = np.linspace(individual_even_load_data['avg_accuracy'].min(),individual_even_load_data['avg_accuracy'].max(),100)*res.params[1] + res.params[0],
                    mode='lines', opacity = 0.5,
                  line = dict(width = 3.5, color='black', dash = 'longdashdot'),
                    name = '$r^{2}_{partial} = 0.753$',
                   showlegend=False)


layout = go.Layout(title = '',font = dict(size = 12),
                  xaxis = dict(title = 'Accuracy (Loads 3-5)'),
                  yaxis = dict(title = 'Accuracy (Loads 6-8)'), plot_bgcolor = 'rgba(0,0,0,0)')

fig = go.Figure(data = [sec2_trace, sec2_trace_fit], layout = layout)
fig['layout']['annotations']= [dict(x=.6525,y=.625,xref='x',yref='y',text='$r^{2} = 0.569^{*}$', showarrow=False, font=dict(
                family="Times", color='black'
            ))]
fig['layout'].update(width = 400, height = 400, font = dict(family = 'Times,Bold', size = 12, color='black'))
fig.update_xaxes(showline=True, linewidth = 2, tickvals=['',0.6,0.7,0.8,0.9],linecolor = '#AFAFAF', range = [0.5, 0.95], showgrid=True, gridwidth=1.5, gridcolor='#DFDFDF', zeroline=True, zerolinecolor = '#AFAFAF', zerolinewidth = 2.5, nticks = 8)
fig.update_yaxes(showline = True, linewidth = 2, linecolor = '#AFAFAF',range = [0.5, 0.75], showgrid=True, gridwidth=1.5, gridcolor='#DFDFDF', zeroline=True, zerolinecolor = '#AFAFAF', zerolinewidth = 2.5, nticks = 8)
fig.show()
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/consistent_individual_differences_split_by_target_load.png")
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/consistent_individual_differences_split_by_target_load.svg")
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/consistent_individual_differences_split_by_target_load.pdf")
#py.iplot(fig, filename='split half correlations across target loads')

0.5691466972120702


In [11]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           avg_accuracy   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     63.41
Date:                Thu, 21 May 2020   Prob (F-statistic):           2.51e-10
Time:                        10:31:15   Log-Likelihood:                 107.38
No. Observations:                  50   AIC:                            -210.8
Df Residuals:                      48   BIC:                            -206.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1831      0.054      3.387      0.001       0.074       0.292
avg_accuracy_even     0.5204      0.065      7.963      0.000       0.389       0.652
==============================================================================
Omnibus:                        1.863   Durbin-Watson:                   1.305
Prob(Omnibus):                  0.394   Jarque-Bera (JB):                1.020
Skew:                          -0.257   Prob(JB):                        0.600
Kurtosis:                       3.475   Cond. No.                         27.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Check for consistencies when compared across speeds for these ranked individuals

In [12]:
odd_set = [5.0, 6.26]
even_set = [1.57,3.13]
sub_id = individual_ranking['subject_Num'].to_numpy()

even_load_data = behavior_data.filter_by(even_set,'speed (deg/sec)')
individual_even_load_data = even_load_data.groupby('subject_Num',operations = {'avg_accuracy':agg.MEAN('accuracy')})
individual_even_load_data = individual_even_load_data.sort('subject_Num')
individual_even_load_data['rank'] = individual_ranking['rank']
individual_even_load_data = individual_even_load_data.sort('rank')

odd_load_data = behavior_data.filter_by(odd_set,'speed (deg/sec)')
individual_odd_load_data = odd_load_data.groupby('subject_Num',operations = {'avg_accuracy':agg.MEAN('accuracy')})
individual_odd_load_data = individual_odd_load_data.sort('subject_Num')
individual_odd_load_data['rank'] = individual_ranking['rank']
individual_odd_load_data = individual_odd_load_data.sort('rank')

hsb = individual_odd_load_data
hsb['avg_accuracy_even'] = individual_even_load_data['avg_accuracy']
hsb2 = hsb.to_dataframe()

mod = ols("avg_accuracy ~ avg_accuracy_even", data=hsb2)
res = mod.fit()
print(res.rsquared)
sec2_trace = go.Scatter(x = individual_even_load_data['avg_accuracy'].to_numpy(),
                       y = individual_odd_load_data['avg_accuracy'].to_numpy(),
                       mode='markers+text', marker = dict(size = 12, opacity = 0.85, color = 'gray',line = dict(width = 1.5, color='black')),
                       text = individual_even_load_data['rank'].to_numpy(),
                        textfont = dict(size = 7),
                       showlegend=False)

sec2_trace_fit = go.Scatter(x = np.linspace(individual_even_load_data['avg_accuracy'].min(),individual_even_load_data['avg_accuracy'].max(),100),
                   y = np.linspace(individual_even_load_data['avg_accuracy'].min(),individual_even_load_data['avg_accuracy'].max(),100)*res.params[1] + res.params[0],
                    mode='lines', opacity = 0.5,
                  line = dict(width = 3.5, color='black', dash = 'longdashdot'),
                    name = '$r^{2}_{partial} = 0.771$',
                   showlegend=False)


layout = go.Layout(title = '',
                  xaxis = dict(title = 'Accuracy (Low speed)'),
                  yaxis = dict(title = 'Accuracy (High speed)'))

fig = go.Figure(data = [sec2_trace, sec2_trace_fit], layout = layout )
fig['layout']['annotations']= [dict(x=.655,y=.69,xref='x',yref='y',text='$r^{2} = 0.775^{*}$', showarrow=False, font=dict(
                family="Times", color='black'
            ))]
fig['layout'].update(width = 400, height = 400, font = dict(family = "Times, Bold", size = 12, color = 'black'), plot_bgcolor = 'rgba(0,0,0,0)')
fig.update_xaxes(showgrid=True, showline=True, tickvals = ['',0.6, 0.7, 0.8, 0.9],linewidth = 2, linecolor = '#AFAFAF', range = [0.5, 0.95], gridwidth=1.5, gridcolor='#DFDFDF', zeroline=True, zerolinecolor = '#AFAFAF', zerolinewidth = 2.5, nticks = 6)
fig.update_yaxes(showgrid=True, showline=True, linewidth = 2, linecolor = '#AFAFAF', range = [0.5, 0.75],gridwidth=1.5, gridcolor='#DFDFDF', zeroline=True, zerolinecolor = '#AFAFAF', zerolinewidth = 2.5, nticks = 6)
fig.show()
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/consistent_individual_differences_split_by_speed.png")
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/consistent_individual_differences_split_by_speed.svg")
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/consistent_individual_differences_split_by_speed.pdf")
#py.iplot(fig, filename='split half correlations across speeds')

0.7758722970900835


In [28]:
avg_acc_data = behavior_data.groupby(['subject_Num'], operations = {'avg_acc':agg.MEAN('accuracy')})
avg_acc_data = avg_acc_data.sort(['subject_Num'])
avg_acc_data.explore()

Materializing SFrame

## Split half correlations across participants

In [33]:
dataframe = tc.SFrame()
dataframe['first_half'] = avg_acc_data[0:25]['avg_acc']
dataframe['second_half'] = avg_acc_data[25:]['avg_acc']
data_frame_pd = dataframe.to_dataframe()
mod = ols("first_half ~ second_half", data=data_frame_pd)
res = mod.fit()
print(res.rsquared)

0.008320122241024852


<a id = section 3></a>
# [Analysis using number of targets chosen](#section 3) 


## Group the behavioral data by target load

In [14]:
target_data = behavior_data.groupby('num_targets',operations = {'avg_num_targets_chosen':agg.MEAN('num_targets_chosen'),
                                                               'std_num_targets_chosen':agg.STD('num_targets_chosen')})
target_data = target_data.sort('num_targets')

## Plot the grouped data

In [15]:
sec3_trace_before = go.Scatter(x = target_data['num_targets'].to_numpy(),
                   y = target_data['avg_num_targets_chosen'].to_numpy() + target_data['std_num_targets_chosen'].to_numpy(),
                   mode = 'lines',
                   line = dict(color = 'beige'), showlegend=False)
sec3_trace = go.Scatter(x = target_data['num_targets'].to_numpy(),
                  y = target_data['avg_num_targets_chosen'].to_numpy(),
                  mode='markers+lines',
                  line = dict(width = 2,
                             color = 'black'),
                  fill = 'tonexty',
                  fillcolor = 'lightgray', name = 'avg')
sec3_trace_after = go.Scatter(x = target_data['num_targets'].to_numpy(),
                   y = target_data['avg_num_targets_chosen'].to_numpy() - target_data['std_num_targets_chosen'].to_numpy(),
                   mode = 'lines',
                   line = dict(color = 'beige'),
                   fill = 'tonexty',
                   fillcolor='lightgray',name='std')

layout = go.Layout(title = 'avg. number of targets chosen vs. target load',
                  xaxis = dict(title = 'target load', range = [2,9]),
                  yaxis = dict(title = 'avg. number of targets chosen'))

fig = go.Figure(data = [sec3_trace_before, sec3_trace, sec3_trace_after], layout = layout )
iplot(fig, filename='MOT_avg_num_targets_chosen_vs_target_load')

<a id = section 3.1></a>
# [Subject wise analysis of the number of targets chosen:](#section 3.1)  Appendix to [Section 3](#section 3)

## Plot the subject ranks vs avg. number of targets chosen for each target load

* For each target load, Plot the overall individual ranks against the avg. number of targets chosen (y axis in the above figure)
* Ideally we should expect a negative slope : Best performers (low rank) on an avg. should track more number of targets

In [16]:
sec2_trace = []
for target in subject_data['num_targets'].unique().sort():
    rank_data = subject_data[subject_data['num_targets']==target]
    rank_data = rank_data.sort('subject_Num')
    rank_data['rank'] = individual_ranking['rank']
    sec2_trace.append(go.Scatter(x = rank_data['rank'].to_numpy(),
                           y = rank_data['avg_num_targets_chosen'].to_numpy(),
                           mode='markers',
                           line=dict(width = 2),
                           name = 'Target load : '+ str(target)))

fig = tools.make_subplots(rows=2, cols=3)
fig.add_trace(sec2_trace[0],1,1)
fig.add_trace(sec2_trace[1],1,2)
fig.add_trace(sec2_trace[2],1,3)
fig.add_trace(sec2_trace[3],2,1)
fig.add_trace(sec2_trace[4],2,2)
fig.add_trace(sec2_trace[5],2,3)



fig['layout'].update(height=700, width=1000, margin = dict(l=130, b=130, r=80, t=150), font = dict(size=15))
fig['layout']['legend'].update(traceorder='normal')

fig['layout']['yaxis4'].update(title='avg num. targets chosen')
fig['layout']['xaxis5'].update(title='overall ranked individual performance in the experiment')


iplot(fig, filename='subejct ranks vs. num targets chosen')    

/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



<a id = section 3.2></a>
# [Proportion of number of targets selected for each target load:](#section 3.2)  

In [17]:
def compute_sum(x):
    sum_counts = numT_select_by_target_load[numT_select_by_target_load['num_targets']==x['num_targets']]['count'].sum()
    return x['count']/sum_counts

In [18]:
numT_select_by_target_load = behavior_data.groupby(['num_targets','num_targets_chosen'],operations = {'count':agg.COUNT('num_targets_chosen')})
numT_select_by_target_load['avg_propT'] = numT_select_by_target_load.apply(lambda x: compute_sum(x))
numT_select_by_target_load = numT_select_by_target_load.sort(['num_targets','num_targets_chosen'])
numT_select_by_target_load.explore()

Materializing SFrame

In [20]:
trace = []
import colorlover as cl
target_load_colors = cl.scales['6']['qual']['Set2']
for i,target in enumerate(numT_select_by_target_load['num_targets'].unique().sort()):
    target_load_data = numT_select_by_target_load[numT_select_by_target_load['num_targets']==target]
    trace.append(go.Scatter(x = target_load_data['num_targets_chosen'].to_numpy(),
                           y = target_load_data['avg_propT'].to_numpy(),
                           name = 'Target load:' + str(target),
                           mode='markers+text+lines',
                           line = dict(shape='spline',width = 3, color = target_load_colors[i]),
                           text = str(target),
                           showlegend=False, 
                           marker = dict(size = 12, line = dict(color='black', width = 1.5), color = target_load_colors[i])))
    
layout = go.Layout(title = '',
                  xaxis = dict(title = 'Number of Targets Accurately Chosen'),
                  yaxis = dict(title = 'Frequency'))

fig = go.Figure(data = trace, layout = layout )
#fig['layout'].update(width = 900, height = 600, )
fig['layout'].update(width = 400, height = 400, font = dict(family = "Times,Bold", size = 12, color = 'black'), plot_bgcolor = 'rgba(0,0,0,0)', paper_bgcolor = 'rgba(0,0,0,0)')
fig.update_xaxes(showgrid=True, showline=True, linewidth = 2, linecolor = '#AFAFAF', gridwidth=1.5, gridcolor='#DFDFDF', zeroline=False, zerolinecolor = '#AFAFAF', zerolinewidth = 2.5, nticks = 6)
fig.update_yaxes(showgrid=True, showline=True, linewidth = 2, linecolor = '#AFAFAF', gridwidth=1.5, gridcolor='#DFDFDF', zeroline=False, zerolinecolor = '#AFAFAF', zerolinewidth = 2.5, nticks = 6)
#fig.show()
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/num_targetsChosen_as_function_of_load.png")
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/num_targetsChosen_as_function_of_load.svg")
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/num_targetsChosen_as_function_of_load.pdf")
#py.iplot(fig, filename='split half correlations across speeds')
iplot(fig, filename='Number of targets chosen as a function of target load')

<a id = section 3.3></a>
# [Centroid Model : Number of targets chosen as a function of target load](#section 3.3)

In [21]:
import turicreate as tc
destPath='./data/MOT_correspondences_replication/'
centroid_model_data = tc.SFrame.read_json(destPath+'correspondences_sub_20_Hz_Centroid.json')

Parsing JSON records from /Volumes/My Passport/Github/MOT_bayesian_model/data/MOT_correspondences_replication/correspondences_sub_20_Hz_Centroid.json

Successfully parsed an SArray of 12000 elements from the JSON file /Volumes/My Passport/Github/MOT_bayesian_model/data/MOT_correspondences_replication/correspondences_sub_20_Hz_Centroid.json

In [22]:
centroid_model_data['num_targets_chosen'] = centroid_model_data.apply(lambda x: int(x['num_targets']*x['predicted_accuracy']))
centroid_model_data.explore()

Materializing SFrame

In [23]:
def compute_sum_centroid(x):
    sum_counts = centroid_numT_select_by_target_load[centroid_numT_select_by_target_load['num_targets']==x['num_targets']]['count'].sum()
    return x['count']/sum_counts

In [24]:
centroid_numT_select_by_target_load = centroid_model_data.groupby(['num_targets','num_targets_chosen'],operations = {'count':agg.COUNT('num_targets_chosen')})
centroid_numT_select_by_target_load['avg_propT'] = centroid_numT_select_by_target_load.apply(lambda x: compute_sum_centroid(x))
centroid_numT_select_by_target_load = centroid_numT_select_by_target_load.sort(['num_targets','num_targets_chosen'])
centroid_numT_select_by_target_load.explore()

Materializing SFrame

In [25]:
trace = []
import colorlover as cl
target_load_colors = cl.scales['6']['qual']['Set2']
for i,target in enumerate(centroid_numT_select_by_target_load['num_targets'].unique().sort()):
    target_load_data = centroid_numT_select_by_target_load[centroid_numT_select_by_target_load['num_targets']==target]
    trace.append(go.Scatter(x = target_load_data['num_targets_chosen'].to_numpy(),
                           y = target_load_data['avg_propT'].to_numpy(),
                           name = 'Target load:' + str(target),
                           mode='markers+text+lines',
                           line = dict(shape='spline',width = 3, color = target_load_colors[i]),
                           text = str(target),
                           showlegend=False, 
                           marker = dict(size = 18, opacity = 1.0, line = dict(color='black', width = 1.5), color = target_load_colors[i])))
    
layout = go.Layout(title = '',
                  xaxis = dict(title = 'Number of Targets Accurately Chosen'),
                  yaxis = dict(title = 'Frequency'))

fig = go.Figure(data = trace, layout = layout )
#fig['layout'].update(width = 900, height = 600, )
fig['layout'].update(width = 500, height = 500, font = dict(size = 14, family = 'Times,Bold', color='black'), plot_bgcolor = 'rgba(0,0,0,0)', paper_bgcolor = 'rgba(0,0,0,0)')
fig.update_xaxes(showgrid=True, showline=True, linewidth = 2, linecolor = '#AFAFAF', gridwidth=1.5, gridcolor='#DFDFDF', zeroline=False, zerolinecolor = '#AFAFAF', zerolinewidth = 2.5, nticks = 6)
fig.update_yaxes(showgrid=True, showline=True, linewidth = 2, linecolor = '#AFAFAF', gridwidth=1.5, gridcolor='#DFDFDF', zeroline=False, zerolinecolor = '#AFAFAF', zerolinewidth = 2.5, nticks = 6)
fig.show()
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/Centroid_model_num_targetsChosen_as_function_of_load.png")
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/Centroid_model_num_targetsChosen_as_function_of_load.svg")
fig.write_image("/Users/supadhy6/Dropbox/Shared with Adi/Results_images/Centroid_model_num_targetsChosen_as_function_of_load.pdf")
#py.iplot(fig, filename='split half correlations across speeds')
#py.iplot(fig, filename='Number of targets chosen as a function of target load')